# Задача #1: Метод разделяющей гиперплоскости

In [ ]:
def cutting_plane(x_0, A, b, c, k=10):
    """
    Решает задачу LP (с^Tx -> min, Ax<=b, x>=0) с помощью метода разделяющей гиперплоскости.
    Начальное приближение задается многогранником Ax<=b, x>=0.
    
    Args:
        x_0: ndarray(n) -- начальное приближение
        A: ndarray(m, n)
        b: ndarray(m)
        c: ndarray(n)
        k: int -- число итераций
    Returns:
        [(p_1, d_1, x_1), ..., (p_k, d_k, x_k)] -- список добавленных уравнений вида p_lx<=b_l и соответствующее приближение x_l.
    """
    pass

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
import os
import sys
module_path = os.path.abspath(os.path.join('..', '..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from animation_utils.animation import animate_list

In [ ]:
def get_line(x1, x2):
    a = x1[1] - x2[1]
    b = x2[0] - x1[0]
    c = a * x1[0] + b * x1[1]
    return a, b, c

vertices = [(2.0, 2.0), (1.9, 3.0), (2.5, 4.0), (4.0, 4.2), (4.7, 3.5), (4.5, 1.5), (3.5, 1.0), (2.0, 2.0)]
A = []
b = []

for i in range(len(vertices) - 1):
    a_, b_, c_ = get_line(vertices[i], vertices[i + 1])
    A.append([a_, b_])
    b.append(c_)
A = np.array(A)
b = np.array(b)
direction = np.array([-2, -1]) # c

In [ ]:
def fix_scaling(ax=None):
    if not ax:
        xlim = plt.xlim()
        ylim = plt.ylim()
        d1 = xlim[1] - xlim[0]
        d2 = ylim[1] - ylim[0]
        if d1 > d2:
            plt.ylim((ylim[0] - (d1-d2) / 2, ylim[1] + (d1-d2) / 2))
        else:
            plt.xlim((xlim[0] + (d1-d2) / 2, xlim[1] - (d1-d2) / 2))
    else:
        xlim = ax.get_xlim()
        ylim = ax.get_ylim()
        d1 = xlim[1] - xlim[0]
        d2 = ylim[1] - ylim[0]
        if d1 > d2:
            ax.set_ylim((ylim[0] - (d1-d2) / 2, ylim[1] + (d1-d2) / 2))
        else:
            ax.set_xlim((xlim[0] + (d1-d2) / 2, xlim[1] - (d1-d2) / 2))

In [ ]:
def intersection(a, b, c, d):
    return np.linalg.solve(np.array([a, c]), np.array([b, d]))

In [ ]:
from scipy.spatial import ConvexHull

def get_vertices(A, b, P, d):
    vertices = []
    #print(A, b, P, d)
    for i in range(b.shape[0]):
        for j in range(i):
            try:
                _x = intersection(A[i], b[i], A[j], b[j])
            except np.linalg.LinAlgError:
                continue
            if np.min(np.less_equal(A @ _x - 1e-6, b)) and np.min(np.less_equal(P @ _x - 1e-6, d)) and np.min(np.less_equal(np.zeros_like(_x), _x)):
                vertices.append(_x)
    for i in range(d.shape[0]):
        for j in range(i):
            try:
                _x = intersection(P[i], d[i], P[j], d[j])
            except np.linalg.LinAlgError:
                continue
            if np.min(np.less_equal(A @ _x - 1e-6, b)) and np.min(np.less_equal(P @ _x - 1e-6, d)) and np.min(np.less_equal(np.zeros_like(_x), _x)):
                vertices.append(_x)   
                
    for i in range(b.shape[0]):
        for j in range(d.shape[0]):
            try:
                _x = intersection(A[i], b[i], P[j], d[j])
            except np.linalg.LinAlgError:
                continue            
            if np.min(np.less_equal(A @ _x - 1e-6, b)) and np.min(np.less_equal(P @ _x - 1e-6, d)) and np.min(np.less_equal(np.zeros_like(_x), _x)):
                vertices.append(_x)
    #plt.scatter([x for x, y in vertices], [y for x, y in vertices])
    #print('Ver', vertices)
    return np.array(vertices)[ConvexHull(vertices).vertices]    

In [ ]:
get_vertices(A, b, A, b)

In [ ]:
def cutting_plane_step(x, A, b, P, d):
    fig, ax = plt.subplots(figsize=(10, 10))

    hull = get_vertices(A, b, P, d)
    
    ax.fill([x for x, y in hull], [y for x, y in hull], color='grey', alpha=0.5)
    ax.plot([x for x, y in vertices], [y for x, y in vertices])
    
    ax.scatter([x[0]], [x[1]], color='black')
    #fix_scaling(ax)
    ax.axis('off')
    plt.close(fig)
    return fig

In [ ]:
steps = cutting_plane(np.array([3.5, 2.8]), A, b, np.array([-2, -1]), k = 30)

In [ ]:
result = []
P = []
d = []
for P_i, d_i, x_i in steps:
    P.append(P_i)
    d.append(d_i)
    result.append(cutting_plane_step(x_i, A, b, np.array(P), np.array(d)))

animate_list(result);